In [1]:
"""
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
%cd drive/My Drive/EAD_1
"""

Mounted at /content/drive


In [55]:
#Read Dataset
import pandas as pd
df=pd.read_csv(r"EAD_Final_data.csv")

In [56]:
df

,Place_of_residence_partner_man,Place_of_birth_partner_man,Age_partner_man,Monthly_income_partner_man_weight,Occupation_partner_man,Place_of_birth_partner_woman,Age_partner_woman,Place_of_residence_partner_woman,Occupation_partner_woman,Monthly_income_partner_woman_weight,Level_of_education_partner_man,Level_of_education_partner_woman,Custody
0,XALAPA-ENRIQUEZ,XALAPA - ENRIQUEZ,25,1000.0,EMPLOYEE,XALAPA-ENRIQUEZ,25,OAXACA,MUSIC TEACHER,43000.0,PROFESSIONAL,PRIMARY,DAD
1,CHICONTEPEC,SAN RAFAEL,35,900.0,GENERAL DOCTOR,SOLITUDE OF BENDING,42,FLAT,RADIO PRODUCER,13569.0,OTHER,OTHER,OTHER
2,SAN ROMAN,RAFAEL LUCIO,62,9000.0,CLINICAL CHEMICAL,POISSY,39,VILLA JUANITA,REPORTER,8500.0,PROFESSIONAL,HIGH SCHOOL,DAD
3,XALAPA-ENRIQUEZ,XALAPA - ENRIQUEZ,28,6000.0,MECHANICAL,XALAPA-ENRIQUEZ,26,XALAPA-ENRIQUEZ,EDUCATOR,7000.0,HIGH SCHOOL,PROFESSIONAL,MOTHER
4,PUEBLA,PUEBLA,44,20000.0,EMPLOYEE,XALAPA-ENRIQUEZ,41,XALAPA-ENRIQUEZ,HOUSEWIFE,1200.0,PROFESSIONAL,PREPARATORY,MOTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7918,XALAPA-ENRIQUEZ,TAMPICO,52,12000.0,EMPLOYEE,TEPEJIL DEL RIO,45,XALAPA-ENRIQUEZ,BUSINESSMAN,20000.0,PRIMARY,PREPARATORY,MOTHER
7919,INDEPENDENCE,RINCONADA,68,10280.0,SIGNWRITER,ACAPULCO,77,OAXACA,LABORATORY,3300.0,PROFESSIONAL,PROFESSIONAL,MOTHER
7920,XALAPA-ENRIQUEZ,WORKER,28,8700.0,CHEMICAL,XALAPA-ENRIQUEZ,25,XALAPA-ENRIQUEZ,LIC. BUSINESS ADMINISTRATION,1070.0,PRIMARY,PROFESSIONAL,MOTHER
7921,XALAPA-ENRIQUEZ,FRANCISCO SARABIA,47,3545.0,EMPLOYEE,XALAPA-ENRIQUEZ,34,XALAPA-ENRIQUEZ,HOUSEWIFE,3900.0,PREPARATORY,HIGH SCHOOL,MOTHER


In [6]:
#Encoding the Training Data
df["Place_of_birth_partner_man"] = df["Place_of_birth_partner_man"].astype('category')
df["Place_of_birth_partner_man_cat"] = df["Place_of_birth_partner_man"].cat.codes
df["Place_of_birth_partner_woman"] = df["Place_of_birth_partner_woman"].astype('category')
df["Place_of_birth_partner_woman_cat"] = df["Place_of_birth_partner_woman"].cat.codes
df['Place_of_residence_partner_man']=df['Place_of_residence_partner_man'].astype('category')
df['Place_of_residence_partner_man_cat']=df['Place_of_residence_partner_man'].cat.codes
df['Place_of_residence_partner_woman']=df['Place_of_residence_partner_man'].astype('category')
df['Place_of_residence_partner_woman_cat']=df['Place_of_residence_partner_man'].cat.codes
df['Occupation_partner_man']=df['Occupation_partner_man'].astype('category')
df['Occupation_partner_man_cat']=df['Occupation_partner_man'].cat.codes
df['Occupation_partner_woman']=df['Occupation_partner_woman'].astype('category')
df['Occupation_partner_woman_cat']=df['Occupation_partner_woman'].cat.codes
df['Level_of_education_partner_man']=df['Level_of_education_partner_man'].astype('category')
df['Level_of_education_partner_man_cat']=df['Level_of_education_partner_man'].cat.codes
df['Level_of_education_partner_woman']=df['Level_of_education_partner_woman'].astype('category')
df['Level_of_education_partner_woman_cat']=df['Level_of_education_partner_woman'].cat.codes
df['Custody']=df['Custody'].astype('category')
df['Custody_cat']=df['Custody'].cat.codes

In [10]:
#Splitting the Dataset
X=df[['Place_of_birth_partner_man_cat',
       'Place_of_birth_partner_woman_cat',
       'Place_of_residence_partner_man_cat',
       'Place_of_residence_partner_woman_cat', 'Occupation_partner_man_cat',
       'Occupation_partner_woman_cat', 'Level_of_education_partner_man_cat',
       'Level_of_education_partner_woman_cat','Age_partner_woman','Age_partner_man']]
y=df["Custody_cat"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=7)

In [12]:
#Divide the Data into Data for Teachers and Students
student_data=X_test
student_data_y=y_test
teacher_data=X_train
teacher_data_y=y_train

In [16]:
#Partitioning the Dataset into 6 partition Teachers
dataframes_teacher={}
dataframes_teacher_y={}
chunk = 924
def divide_chunks(df,chunk,dfs):
  for n in range((df.shape[0] // chunk + 1)):
      df_temp = df.iloc[n*chunk:(n+1)*chunk]
      df_temp = df_temp.reset_index(drop=True)
      dfs[n] = df_temp
divide_chunks(teacher_data,chunk,dataframes_teacher)
divide_chunks(teacher_data_y,chunk,dataframes_teacher_y)
def append_last(dfs):
  dfs[len(dfs)-2]=dfs[len(dfs)-2].append(dfs[len(dfs)-1])
  del dfs[len(dfs)-1]
append_last(dataframes_teacher)
append_last(dataframes_teacher_y)

In [18]:
#Training the 6 Teachers with the Poly Kernel
from sklearn import svm
clf0 = svm.SVC(kernel='poly')
clf0.fit(dataframes_teacher[0].values,dataframes_teacher_y[0])
clf1 = svm.SVC(kernel='poly')
clf1.fit(dataframes_teacher[1].values,dataframes_teacher_y[1])
clf2 = svm.SVC(kernel='poly')
clf2.fit(dataframes_teacher[2].values,dataframes_teacher_y[2])
clf3 = svm.SVC(kernel='poly') 
clf3.fit(dataframes_teacher[3].values,dataframes_teacher_y[3])
clf4 = svm.SVC(kernel='poly') 
clf4.fit(dataframes_teacher[4].values,dataframes_teacher_y[4])
clf5 = svm.SVC(kernel='poly') 
clf5.fit(dataframes_teacher[5].values,dataframes_teacher_y[5])

SVC(kernel='poly')

In [20]:
#Taking Predictions from 6 Teachers for Student Data
df_inp=student_data
t=[]
for i in range(len(df_inp)):
  l=[]
  l.append(clf0.predict([df_inp.iloc[i]])[0])
  l.append(clf2.predict([df_inp.iloc[i]])[0])
  l.append(clf1.predict([df_inp.iloc[i]])[0])
  l.append(clf3.predict([df_inp.iloc[i]])[0])
  l.append(clf4.predict([df_inp.iloc[i]])[0])
  l.append(clf5.predict([df_inp.iloc[i]])[0])
  t.append(l)

In [22]:
#Finding the Aggregate Noisy Max of the predicted labels
def score(data, option):
    x = pd.Series(data)
    return x.value_counts()[option]/1000
import numpy as np
def report_noisy_max(x, R, u, sensitivity, epsilon):
    scores = [u(x, r) for r in R]
    noisy_scores = [np.exp(epsilon * score / (2 * sensitivity)) for score in scores]
    max_idx = np.argmax(noisy_scores)
    return R[max_idx]
r = [report_noisy_max(t[i],list(set(t[i])), score, 1, 1) for i in range(len(t))]
pd.Series(r).value_counts()

1    1601
2     627
0     148
dtype: int64

In [23]:
student_data_y.value_counts()

1    1101
0     642
2     633
Name: Custody_cat, dtype: int64

In [40]:
#Training the Student with no Differential Privacy
X_nodp=student_data[['Place_of_birth_partner_man_cat',
       'Place_of_birth_partner_woman_cat',
       'Place_of_residence_partner_man_cat',
       'Place_of_residence_partner_woman_cat', 'Occupation_partner_man_cat',
       'Occupation_partner_woman_cat', 'Level_of_education_partner_man_cat',
       'Level_of_education_partner_woman_cat','Age_partner_woman','Age_partner_man']]
y_nodp=student_data_y

In [41]:
#Splitting the Student Data without Differential Privacy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_dp, y_dp, test_size=0.3,random_state=7)

In [42]:
#Training SVM with linear kernel without Differential Privacy
%%time
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

CPU times: user 7min 18s, sys: 341 ms, total: 7min 19s
Wall time: 7min 17s


In [44]:
#Getting Accuracy of the Model without Differential Privacy
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

(0.3810949955177094, 0.37461788655265194, 0.34884312229849995, None)
Accuracy: 0.47124824684431976


In [45]:
#Getting Confusion Matrix of the Model without Differential Privacy
print("Confusion Matrix\n",metrics.confusion_matrix(y_test, y_pred))

Confusion Matrix
 [[ 33 119  29]
 [ 28 280  33]
 [ 34 134  23]]


In [46]:
sy_test=y_test

In [47]:
#Splitting Student Data with Differential Privacy
from sklearn.model_selection import train_test_split
X_dp=student_data[['Place_of_birth_partner_man_cat',
       'Place_of_birth_partner_woman_cat',
       'Place_of_residence_partner_man_cat',
       'Place_of_residence_partner_woman_cat', 'Occupation_partner_man_cat',
       'Occupation_partner_woman_cat', 'Level_of_education_partner_man_cat',
       'Level_of_education_partner_woman_cat','Age_partner_woman','Age_partner_man']]
y_dp=r
X_train, X_test, y_train, y_test = train_test_split(X_dp, y_dp, test_size=0.3,random_state=7)

In [49]:
#Training SVM with linear kernel without Differential Privacy
%%time
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

CPU times: user 47.6 s, sys: 44.9 ms, total: 47.6 s
Wall time: 47.4 s


In [53]:
#Getting Accuracy of the Model without Differential Privacy
from sklearn import metrics
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
print("Accuracy:",metrics.accuracy_score(sy_test, y_pred))

(0.6904970329775861, 0.6486965987805116, 0.6645256100720559, None)
Accuracy: 0.4838709677419355


In [54]:
#Getting Confusion Matrix of the Model without Differential Privacy
print("Confusion Matrix\n",metrics.confusion_matrix(sy_test, y_pred))

Confusion Matrix
 [[ 10 107  64]
 [ 10 276  55]
 [  8 124  59]]


In [57]:
#Saving SVM Model with Differential Privacy 
import pickle
filename = 'SVMFinal.pkl'
pickle.dump(clf, open(filename, 'wb'))
 
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, sy_test)
print(result)

0.4838709677419355


In [64]:
#Predict for a single record
loaded_model.predict([[654, 94, 183, 183, 100, 94, 1, 2, 34, 36]])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  "X does not have valid feature names, but"


array([1], dtype=int8)